This is a tutorial pyflowline notebook.
First, let's load some Python libraries 

In [1]:
import sys

import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(format='%(asctime)s %(message)s')
logging.warning('is the time Pyflowline simulation started.')

2022-03-03 17:14:34,460 is the time Pyflowline simulation started.


The pyflowline python package uses the OOP approach to manage configuration settings.
Set up the case object. 
Set up some core keywords.

In [2]:
sMesh_type = 'mpas'
iCase_index = 1
dResolution_meter=5000
sDate='20220301'

We provide two options to set up a simulation case. First, from scratch (iFlag_option: 1); second, using a json file (iFlag_option = 2). For details of the json file format, please read the documentation.
We will use option 1 in this tutorial.

In [3]:
from pathlib import Path
from os.path import realpath
from pyflowline.classes.pycase import flowlinecase
from pyflowline.pyflowline_generate_template_configuration_json_file import pyflowline_generate_template_configuration_json_file
from pyflowline.pyflowline_read_model_configuration_file import pyflowline_read_model_configuration_file

iFlag_option = 1
sPath = str( Path().resolve() )
sPath_data = realpath( sPath +  '/../data/susquehanna' )
print(sPath)

if iFlag_option ==1:
    
    sFilename_configuration_in = realpath(sPath +  '/../tests/configurations/template.json' )
    oPyflowline = pyflowline_generate_template_configuration_json_file(sFilename_configuration_in,sPath_data, iFlag_use_mesh_dem_in = 1, sMesh_type_in=sMesh_type, iCase_index_in = iCase_index, sDate_in = sDate)
else: 
    if iFlag_option == 2:
        #an example configuration file is provided with the repository, 
        #but you need to update this file based on your own case study
        
        if sMesh_type=='hexagon':
            sFilename_configuration_in = realpath( sPath +  '/../tests/configurations/pyflowline_susquehanna_hexagon.json' ) 
        else:
            if sMesh_type=='square':
                sFilename_configuration_in = realpath(sPath +  '/../tests/configurations/pyflowline_susquehanna_square.json' )
            else:
                if sMesh_type=='latlon':
                    sFilename_configuration_in = realpath( sPath +  '/../tests/configurations/pyflowline_susquehanna_latlon.json' )
                else:
                    sFilename_configuration_in =realpath( sPath +  '/../tests/configurations/pyflowline_susquehanna_mpas.json' )
        
        oPyflowline = pyflowline_read_model_configuration_file(sFilename_configuration_in)
        

/Users/liao313/workspace/python/pyflowline_icom/notebooks


In [4]:
print(oPyflowline.tojson())


{
    "dLatitude_bot": -90.0,
    "dLatitude_top": 90.0,
    "dLongitude_left": -180.0,
    "dLongitude_right": 180.0,
    "dResolution_degree": 0.5,
    "dResolution_meter": 50000.0,
    "iCase_index": 1,
    "iFlag_flowline": 1,
    "iFlag_save_mesh": 1,
    "iFlag_standalone": 1,
    "iFlag_use_mesh_dem": 0,
    "iFlag_use_shapefile_extent": 0,
    "iMesh_type": 4,
    "nOutlet": 1,
    "sCase": "pyflowline20220301001",
    "sDate": "20220301",
    "sFilename_basins": "/Users/liao313/workspace/python/pyflowline_icom/tests/configurations/template_basins.json",
    "sFilename_mesh": "/Users/liao313/workspace/python/pyflowline_icom/data/susquehanna/output/pyflowline20220301001/mpas.json",
    "sFilename_mesh_info": "/Users/liao313/workspace/python/pyflowline_icom/data/susquehanna/output/pyflowline20220301001/mpas_mesh_info.json",
    "sFilename_mesh_netcdf": "/Users/liao313/workspace/python/pyflowline_icom/data/susquehanna/input/lnd_cull_mesh.nc",
    "sFilename_model_configuration": "

After the case object was created, we can run the mainly three types of operations. 
First, we can preprocess the flowline.

Before the operation, we can visualize the original or raw flowline dataset. For most visualization, all you need to provide is a keyword, a look up table is provided in the documentation.
Be default, all the dataset will be converted to the WGS84 system to maintain consistency.

In [5]:
oPyflowline.aBasin[0].dLatitude_outlet_degree=39.4620
oPyflowline.aBasin[0].dLongitude_outlet_degree=-76.0093


In [6]:
#pyflowline can process multiple basins within one singel run
#the total number of basin is controlled by the nOutlet variable
#convert the raw flowline into geojson in WGS84 system        
oPyflowline.convert_flowline_to_json()

In [7]:

oPyflowline.flowline_simplification()


After the preprocess, we can visualize the final flowline 

Second, we can creata a mesh

In [8]:
print(oPyflowline.sFilename_mesh)

oPyflowline.dLongitude_left= -79
oPyflowline.dLongitude_right= -74.5
oPyflowline.dLatitude_bot= 39.20
oPyflowline.dLatitude_top= 42.8
oPyflowline.mesh_generation()

/Users/liao313/workspace/python/pyflowline_icom/data/susquehanna/output/pyflowline20220301001/mpas.json


 ...]

We can also visualize the mesh:

Last, we can operate the intersect to generate desired flowlines. But we recommend that we run the intersect with some postprocess.

In [9]:


oPyflowline.reconstruct_topological_relationship()


KeyError: 'Illegal field requested in GetField()'

After this step, we can now visualize the final flowline again.

In [ ]:
oPyflowline.analyze()

oPyflowline.export()

In [ ]:
print('Finished')

logging.basicConfig(format='%(asctime)s %(message)s')
logging.warning('is the time Pyflowline simulation finished.')